In [ ]:
import numpy as np
import os
import glob
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd drive/MyDrive/Colab\ Notebooks/scripts

/content/drive/MyDrive/Colab Notebooks/scripts


In [ ]:
from feature_selection import forwardFeatureSelection
from aux_GenLinCFA import prepare_target_binary
from aux_GenLinCFA import prepare_target

In [ ]:
cd ..

/content/drive/MyDrive/Colab Notebooks


## Features averaged on whole basin for each basin

In [ ]:
csv_with_aggreg = "./features_with_aggregations.csv"
all_features = pd.read_csv(csv_with_aggreg).rename(columns={'Unnamed: 0': 'date'})
features_df = all_features.filter(like=basin)

In [ ]:
# merge with target values of Emiliani2
path_target = './csv/'
csv_target = "./csv/" + basin + ".csv"
target = pd.read_csv(csv_target).rename(columns={'Unnamed: 0': 'date'})

# remove dates not available in features
target = target[target['date'] <= all_features['date'].iloc[-1]]

In [ ]:
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

0: cyclostationary_mean_Emiliani1_tg
1: cyclostationary_mean_Emiliani1_rr
2: cyclostationary_mean_Emiliani1_tg_1w
3: cyclostationary_mean_Emiliani1_tg_4w
4: cyclostationary_mean_Emiliani1_tg_8w
5: cyclostationary_mean_Emiliani1_tg_12w
6: cyclostationary_mean_Emiliani1_tg_16w
7: cyclostationary_mean_Emiliani1_tg_24w
8: cyclostationary_mean_Emiliani1_rr_1w
9: cyclostationary_mean_Emiliani1_rr_4w
10: cyclostationary_mean_Emiliani1_rr_8w
11: cyclostationary_mean_Emiliani1_rr_12w
12: cyclostationary_mean_Emiliani1_rr_16w
13: cyclostationary_mean_Emiliani1_rr_24w


# Feature Selection and linear, logistic regression (continuous target)

### Divide features and target in train_val and test

In [ ]:
# features selection done on training and validation set

date_valid_end = '2014-12-31'
date_test_end = '2019-12-31'

features_train_val = features_df.loc[all_features['date'] <= date_valid_end,:].copy()
features_test = features_df.loc[(all_features['date'] > date_valid_end) & (all_features['date'] <= date_test_end),:].copy()
len(features_train_val)

639

In [ ]:
target_train_val = target.loc[target['date'] <= date_valid_end,:]['mean'].to_frame().copy()
target_test = target.loc[(target['date'] > date_valid_end) & (target['date'] <= date_test_end),:]['mean'].to_frame().copy()
len(target_train_val)

639

In [ ]:
scaler_f = StandardScaler()
scaler_t = StandardScaler()

features_train_val = pd.DataFrame(scaler_f.fit_transform(features_train_val.values),
                               index=features_train_val.index, columns=features_train_val.columns)
target_train_val['mean'] = scaler_t.fit_transform(target_train_val)

features_test = pd.DataFrame(scaler_f.fit_transform(features_test.values),
                               index=features_test.index, columns=features_test.columns)
target_test['mean'] = scaler_t.transform(target_test)

### FS algorithm

In [ ]:
res = {
          "delta" : [],
          "numSelected" : [],
          "selectedFeatures" : []
      }
delta = 10
k = 10
res['selectedFeatures'] = forwardFeatureSelection(delta, np.array(features_train_val.values), np.array(target_train_val['mean']), res, delta,1)
selectedFeatures='selectedFeatures'
print(f'\n{res[selectedFeatures]}\n')

----- MI Scores -----
[(9, 0.17041186858054383), (8, 0.1648332049131115), (10, 0.14736930929716657), (11, 0.13629091542369332), (12, 0.11995623648323962), (0, 0.11588360717929823), (1, 0.09218339024471321), (13, 0.07751129197507917), (2, 0.07624477236090975), (6, 0.06284608934165047), (5, 0.051790012755202605), (4, 0.05068230872635781), (3, 0.020999956603970124), (7, 0.016088261985364893)]
Best MI score: 0.17041186858054383
Adding first best original feature: 9
CMI: 0.09720366837934666
CMI: 0.031068280869894882
CMI: 0.0843642866203421
CMI: 0.031961045593324106
CMI: 0.011625703451463548
CMI: 0.023692086720127764
CMI: 0.04381413827358352
CMI: 0.026380305971849677
CMI: 0.058064533284264974
CMI: 0.022484789881361544
CMI: 0.038275839679744406
CMI: 0.010975783225232433
CMI: 0.014648817072766801
Highest CMI score: 0.09720366837934666
Adding original feature: 0
CMI: 0.01099267966924905
CMI: 0.00850854126448658
CMI: 0.002738299382470444
Highest CMI score: 0.01099267966924905
Adding original fea

### Linear, Logistic Regression

In [ ]:
res[selectedFeatures] = res[selectedFeatures][0:5]
res[selectedFeatures]

[9, 0, 2, 10, 8]

In [ ]:
features_dir = "./CMI_features_nocoord/continuous_target/"

chosen_features_train_val = features_train_val[features_train_val.columns[res[selectedFeatures]]]
chosen_features_test = features_test[features_test.columns[res[selectedFeatures]]]

train_val_string = features_dir + basin + '_best5_CMI_trainval.csv'
test_string = features_dir + basin + '_best5_CMI_test.csv'
chosen_features_train_val.to_csv(train_val_string, index=False)
chosen_features_test.to_csv(test_string, index=False)

print('###### Linear Regression ######')
lin_regr = LinearRegression()
# CMI best 5
lin_regr.fit(chosen_features_train_val.values, target_train_val.values.ravel())
print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(chosen_features_train_val.values, target_train_val.values.ravel()),3))
print("Test R2 linear regression CMI best 5: ", round(lin_regr.score(chosen_features_test.values, target_test.values.ravel()),3), "\n")


target_train_val_binary = target_train_val.apply(lambda x: np.sign(x), axis=1)
target_test_binary = target_test.apply(lambda x: np.sign(x), axis=1)

print('###### Binary Classification ######')

log_regr = LogisticRegression(solver='lbfgs', max_iter=1000, random_state = 42)
log_regr.fit(chosen_features_train_val.values, target_train_val_binary.values.ravel())
print("Train accuracy logregr CMI best 5 : ", round(log_regr.score(chosen_features_train_val.values, target_train_val_binary.values.ravel()),3))
print("Test accuracy logregr CMI best 5 : ", round(log_regr.score(chosen_features_test.values, target_test_binary.values.ravel()),3), "\n")

###### Linear Regression ######
Train R2 linear regression CMI best 5:  0.351
Test R2 linear regression CMI best 5:  0.321 

###### Binary Classification ######
Train accuracy logregr CMI best 5 :  0.726
Test accuracy logregr CMI best 5 :  0.732 



### Multi Task scores regression

In [ ]:
# for linear regression

def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):

    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)

    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()

    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LinearRegression()
    model_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())

    for basin in clust_basins:
        print(basin)
        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        print(r2_score(targets_df_test[basin].values.ravel(), res))

In [ ]:
### continuous targets
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
path_targets = "./csv/"
targets_df_train = pd.DataFrame()
targets_df_val = pd.DataFrame()
targets_df_test = pd.DataFrame()
targets_df_trainVal = pd.DataFrame()

for basin in basins:
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01',
                                                                                     max_test='2020-01-01', path=path_targets+basin+'.csv',
                                                                                     window_size = 1)
    targets_df_train[basin] = target_df_train.mean_std
    targets_df_val[basin] = target_df_val.mean_std
    targets_df_test[basin] = target_df_test.mean_std
    targets_df_trainVal[basin] = target_df_trainVal.mean_std

In [ ]:
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona',
          'Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

### CMI best5 features
#path_features = './NonLinCFA/temp_prec_internal_ordering/dyn_eps_lin/'
path_features = "./CMI_features_nocoord/continuous_target/"

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_best5_CMI_trainval.csv').iloc[:411,:]
    val_temp = pd.read_csv(path_features+basin+'_best5_CMI_trainval.csv').iloc[411:639,:]
    val_temp.reset_index(drop=True, inplace=True)
    test_temp = pd.read_csv(path_features+basin+'_best5_CMI_test.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
MTL_scores(clust_basins=['Emiliani1','Emiliani2','Garda_Mincio'], df_train=best5_CMI_fulldf_train,
           df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train,
           targets_df_val=targets_df_val, targets_df_test=targets_df_test)

In [ ]:
MTL_scores(clust_basins=['Adda','Lambro_Olona','Oglio_Iseo', 'Ticino'], df_train=best5_CMI_fulldf_train,
           df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train,
           targets_df_val=targets_df_val, targets_df_test=targets_df_test)

In [ ]:
MTL_scores(clust_basins=['Dora','Piemonte_Nord','Piemonte_Sud'], df_train=best5_CMI_fulldf_train,
           df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train,
           targets_df_val=targets_df_val, targets_df_test=targets_df_test)

# Feature Selection and logistic regression (discrete target)

### Divide features and target in train_val and test

In [ ]:
# features selection done on training and validation set

date_valid_end = '2014-12-31'
date_test_end = '2019-12-31'

features_train_val = features_df.loc[all_features['date'] <= date_valid_end,:].copy()
features_test = features_df.loc[(all_features['date'] > date_valid_end) & (all_features['date'] <= date_test_end),:].copy()
len(features_train_val)

639

In [ ]:
target_train_val = target.loc[target['date'] <= date_valid_end,:]['mean'].to_frame().copy()
target_test = target.loc[(target['date'] > date_valid_end) & (target['date'] <= date_test_end),:]['mean'].to_frame().copy()
len(target_train_val)

639

In [ ]:
scaler_f = StandardScaler()

features_train_val = pd.DataFrame(scaler_f.fit_transform(features_train_val.values),
                               index=features_train_val.index, columns=features_train_val.columns)

features_test = pd.DataFrame(scaler_f.fit_transform(features_test.values),
                               index=features_test.index, columns=features_test.columns)

In [ ]:
scaler_t = StandardScaler()

target_train_val['mean'] = scaler_t.fit_transform(target_train_val)
target_test['mean'] = scaler_t.transform(target_test)

In [ ]:
target_train_val = target_train_val.apply(lambda x: np.sign(x), axis=1)
target_test = target_test.apply(lambda x: np.sign(x), axis=1)

### FS algorithm

In [ ]:
res = {
          "delta" : [],
          "numSelected" : [],
          "selectedFeatures" : []
      }
delta = 10
k = 10
res['selectedFeatures'] = forwardFeatureSelection(delta, np.array(features_train_val.values), np.array(target_train_val.values), res, delta,1)
selectedFeatures='selectedFeatures'
print(f'\n{res[selectedFeatures]}\n')

----- MI Scores -----
[(0, 0.07377159577435016), (2, 0.06979123361600668), (8, 0.03617305525235264), (12, 0.031948663470024875), (11, 0.023667337099783064), (13, 0.019895276450792292), (1, 0.01685600404210754), (3, 0.015394266323718177), (5, 0.014150712199026187), (4, 0.011807084007060672), (7, 0.011738250737526889), (9, 0.011650054558383843), (6, 0.003340490036492996), (10, 0.000913150784347241)]
Best MI score: 0.07377159577435016
Adding first best original feature: 0
CMI: 0.010506775821494369
CMI: 0.0125130596333623
CMI: 0.013584757203425168
CMI: 0.007960443051661673
CMI: 0.0017166698189011315
CMI: 0.010069798760638182
Highest CMI score: 0.013584757203425168
Adding original feature: 9
CMI: 0.006086975128703709
CMI: 0.00611108278699822
CMI: 0.019666041993673167
CMI: 0.01337199616991093
CMI: 0.005402312306529988
CMI: 0.0024869256583497368
CMI: 0.004432264352596332
CMI: 0.01408179857710734
Highest CMI score: 0.019666041993673167
Adding original feature: 5
CMI: 0.001332347331132147
CMI: 

### Logistic Regression

In [ ]:
res[selectedFeatures] = res[selectedFeatures][0:5]
res[selectedFeatures]

[0, 9, 5, 12]

In [ ]:
features_dir = "./CMI_features_nocoord/discrete_target/"

chosen_features_train_val = features_train_val[features_train_val.columns[res[selectedFeatures]]]
chosen_features_test = features_test[features_test.columns[res[selectedFeatures]]]

train_val_string = features_dir + basin + '_best5_CMI_trainval.csv'
test_string = features_dir + basin + '_best5_CMI_test.csv'
chosen_features_train_val.to_csv(train_val_string, index=False)
chosen_features_test.to_csv(test_string, index=False)

print('###### Binary Classification ######')

log_regr = LogisticRegression(solver='lbfgs', max_iter=1000, random_state = 42)
log_regr.fit(chosen_features_train_val.values, target_train_val.values.ravel())
print("Train accuracy logregr CMI best 5 : ", round(log_regr.score(chosen_features_train_val.values, target_train_val.values.ravel()),3))
print("Test accuracy logregr CMI best 5 : ", round(log_regr.score(chosen_features_test.values, target_test.values.ravel()),3), "\n")

###### Binary Classification ######
Train accuracy logregr CMI best 5 :  0.648
Test accuracy logregr CMI best 5 :  0.684 



### Multi Task scores classif

In [ ]:
# for binary classification

from sklearn.metrics import accuracy_score
def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):

    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)

    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()

    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LogisticRegression(max_iter = 500)
    model_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())

    for basin in clust_basins:
        print(basin)
        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        print(accuracy_score(targets_df_test[basin].values.ravel(), res))

In [ ]:
### binary targets
basins = ['Emiliani1','Emiliani2','Garda_Mincio', 'Adda', 'Lambro_Olona', 'Ticino',
          'Oglio_Iseo', 'Dora', 'Piemonte_Sud', 'Piemonte_Nord']

path_target = "./csv/"
targets_df_train = pd.DataFrame()
targets_df_val = pd.DataFrame()
targets_df_test = pd.DataFrame()
targets_df_trainVal = pd.DataFrame()

for basin in basins:
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target_binary('',max_train='2010-01-01',
                                                                                            max_val='2015-01-01', max_test='2020-01-01',
                                                                                            path=path_target+basin+'.csv', window_size = 1)
    targets_df_train[basin] = target_df_train.mean_std
    targets_df_val[basin] = target_df_val.mean_std
    targets_df_test[basin] = target_df_test.mean_std
    targets_df_trainVal[basin] = target_df_trainVal.mean_std

In [ ]:
### CMI best5 features
path_features = "./CMI_features_nocoord/discrete_target/"

basins = ['Emiliani1','Emiliani2','Garda_Mincio', 'Adda', 'Lambro_Olona', 'Ticino',
          'Oglio_Iseo', 'Dora', 'Piemonte_Sud', 'Piemonte_Nord']

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_best5_CMI_trainval.csv').iloc[:411,:]
    val_temp = pd.read_csv(path_features+basin+'_best5_CMI_trainval.csv').iloc[411:639,:]
    val_temp.reset_index(drop=True, inplace=True)
    test_temp = pd.read_csv(path_features+basin+'_best5_CMI_test.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
MTL_scores(clust_basins=['Emiliani1','Emiliani2','Garda_Mincio'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

In [ ]:
MTL_scores(clust_basins=['Adda','Lambro_Olona','Oglio_Iseo', 'Ticino'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

In [ ]:
MTL_scores(clust_basins=['Dora', 'Piemonte_Sud', 'Piemonte_Nord'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)